In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split as sk_split
from sklearn import linear_model

np.set_printoptions(threshold='nan')

In [9]:
data = pd.read_csv('datasets/tweets.csv', delimiter=',')

clinton_data = data[data.handle == "HillaryClinton"]["text"].as_matrix()
trump_data = data[data.handle == "realDonaldTrump"]["text"].as_matrix()

clinton_dataset = []
trump_dataset = []

def loadCharsFromTxt(text, dataset):
    image = np.zeros((140, 37))
    words = text.lower().split()
    
    i = 0
    while i < len(words): 
        if "t.co" in words[i]:
            del words[i]
        i += 1
    
    text = " ".join(words)
            
    charCount = 0
    for char in text:
        index = None
        if char.isalpha():
            index = ord(char) - ord('a') + 1
        elif char.isdigit():
            index = ord(char) - ord('0') + 27
        elif char == " ":
            index = 0
                             
        if index:
            image[charCount, index] = 1
            charCount += 1
                    
    dataset.append(image)

for text in clinton_data:
    loadCharsFromTxt(text, clinton_dataset)

for text in trump_data:
    loadCharsFromTxt(text, trump_dataset)
    
clinton_y = np.zeros(len(clinton_dataset))
trump_y = np.full(len(trump_dataset), 1)

x_data = np.concatenate((clinton_dataset, trump_dataset), axis=0)
y_data = np.concatenate((clinton_y, trump_y), axis=0)

x_train, x_test, y_train, y_test = sk_split(x_data, y_data, test_size = 0.25, random_state = 42)

print x_train.shape
print y_train.shape

(4833, 140, 37)
(4833,)


In [61]:
def loadDictionary(text, wc):
    words = text.lower().split()
    for word in words:
        if "t.co" not in word:
            i = 0
            editedWord = word
            while i < len(word):
                if not word[i].isalpha() and not word[i].isdigit():
                    editedWord = word[:i] + word[i+1:]
                i += 1
                
            if word not in wordDictionary:
                wordDictionary[word] = wc
                wc += 1
    
    return wc
                
def bagOfWords(text, dataset, wc):
    bag = np.zeros(wc)
    for word in text.lower().split():
        # ignore t.co links
        if "t.co" not in word:
            i = 0
            
#             ignore punctuation
            editedWord = word
            while i < len(word):
                if not word[i].isalpha() and not word[i].isdigit():
                    editedWord = word[:i] + word[i+1:]
                i += 1

            if editedWord in wordDictionary:
                bag[wordDictionary[editedWord]] += 1
    dataset.append(bag)
    
clintonData = np.array([np.array([text, 0]) for text in clinton_data])
trumpData = np.array([np.array([text, 1]) for text in trump_data])

data = np.concatenate((clintonData, trumpData), axis=0)

x_train, x_test, y_train, y_test = sk_split(data[:, 0], data[:, 1], test_size = 0.25, random_state = 42)

wordDictionary = {}
wordCount = 0
            
for text in x_train:
    wordCount = loadDictionary(text, wordCount)

trainingBags = []
testingBags = []
for text in x_train:
    bagOfWords(text, trainingBags, wordCount)
    
for text in x_test:
    bagOfWords(text, testingBags, wordCount)

model = linear_model.LogisticRegression()
model.fit(trainingBags, y_train)
print model.score(trainingBags, y_train)
print model.score(testingBags, y_test)

0.977860542106
0.891371818746


In [7]:
def bagOfChars(text, dataset, cc):
    bag = np.zeros(cc)
    words = text.lower().split()
    
    i = 0
    while i < len(words): 
        if "t.co" in words[i]:
            del words[i]
        i += 1
    
    text = " ".join(words)
            
    charCount = 0
    for char in text:
        index = None
        if char.isalpha():
            index = ord(char) - ord('a') + 1
        elif char.isdigit():
            index = ord(char) - ord('0') + 27
        elif char == " ":
            index = 0
                             
        if index:
            bag[index] += 1
    
    dataset.append(bag)

clintonData = np.array([np.array([text, 0]) for text in clinton_data])
trumpData = np.array([np.array([text, 1]) for text in trump_data])

data = np.concatenate((clintonData, trumpData), axis=0)

x_train, x_test, y_train, y_test = sk_split(data[:, 0], data[:, 1], test_size = 0.25, random_state = 42)

charCount = 37

trainingBags = []
testingBags = []
for text in x_train:
    bagOfChars(text, trainingBags, charCount)
    
for text in x_test:
    bagOfChars(text, testingBags, charCount)

model = linear_model.LogisticRegression()
model.fit(trainingBags, y_train)
print model.score(trainingBags, y_train)
print model.score(testingBags, y_test)

0.684667908132
0.692116697703


In [59]:
from collections import Counter, defaultdict

# breaks text into words
def text_split(text):
    return re.findall('[a-z0-9]+', text)

# train classifier with training data
def train_classifier(clinton_count, trump_count):
    priors = Counter()
    priors['clinton'] = clinton_count
    priors['trump'] = trump_count
    
    likelihood = defaultdict(Counter)
    total_clinton_training_words = 0 
    total_trump_training_words = 0
    
    for index, tweet in enumerate(x_train):
        for word in tweet.lower().split():
            # ignore t.co links
            if "t.co" not in word:
                i = 0

                # ignore punctuation
                editedWord = word
                while i < len(word):
                    if not word[i].isalpha() and not word[i].isdigit():
                        editedWord = word[:i] + word[i+1:]
                    i += 1

                if int(y_train[index]) == 0:
                    total_clinton_training_words += 1
                    likelihood['clinton'][editedWord] += 1
                else:
                    total_trump_training_words += 1
                    likelihood['trump'][editedWord] += 1

    return(priors, likelihood)

# classifier - uses priors & likelihood to calculate the posterior with test data  
def classify_bayesian(tweet, priors, likelihood):
    # return the class that maximizes the posterior
    max_class = (-1E6, '')
    for category in priors.keys():
        # the number of articles in that category (used to estimate category frequency) 
        p = priors[category]

        # calculate the total number of words for that category (positive or negative)
        # used to estimate word frequency in a category
        n = float(sum(likelihood[category].itervalues()))

        # calculate the posterior by summing the products of word frequency and category 
        # frequency for each word
        # 10,000 is an arbitrary value I found I could scale by such that the least number of
        # "infinite" and "0.0" p-values would be printed and highest accuracy would occur. 
        for word in tweet.lower().split():
            # ignore t.co links
            if "t.co" not in word:
                i = 0

                # ignore punctuation
                editedWord = word
                while i < len(word):
                    if not word[i].isalpha() and not word[i].isdigit():
                        editedWord = word[:i] + word[i+1:]
                    i += 1
                    
                p = 10000 * p * max(1E-10, likelihood[category][editedWord] / n)

        if p > max_class[0]:
            max_class = (p,category)
    return max_class[1]

clintonData = np.array([np.array([text, 0]) for text in clinton_data])
trumpData = np.array([np.array([text, 1]) for text in trump_data])

data = np.concatenate((clintonData, trumpData), axis=0)

x_train, x_test, y_train, y_test = sk_split(data[:, 0], data[:, 1], test_size = 0.15, random_state = 42)

trump_tweet_count = np.count_nonzero(y_train == '1')
clinton_tweet_count = np.count_nonzero(y_train == '0')

(priors, likelihood) = train_classifier(clinton_tweet_count, trump_tweet_count)

num_correct = 0
total_test_pages = len(x_test)

for index, tweet in enumerate(x_test):
    result = classify_bayesian(tweet, priors, likelihood)

    if (result == 'clinton' and int(y_test[index]) == 0) or (result == 'trump' and int(y_test[index]) == 1):
        num_correct += 1

# report data
print "Classified {0} correctly out of {1}; {2} correct\n".format(num_correct, total_test_pages,
    float(num_correct)/total_test_pages)

Classified 898 correctly out of 967; 0.928645294726 correct

